# Preparing(preprocessing data for finetuning)

In [ ]:
def format_input(row):
    return (
        f"FM_NAME_EN: {row['FM_NAME_EN']}, LASTNAME_EN: {row['LASTNAME_EN']}, "
        f"FM_NAME_V1: {row['FM_NAME_V1']}, LASTNAME_V1: {row['LASTNAME_V1']}, "
        f"RLN_TYPE: {row['RLN_TYPE']}, "
        f"Correct_Hin_Surname: {row['Correct_Hin_Surname']}, "
        f"RLN_NAME_EN: {row['RLN_FM_NM_EN']} {row['RLN_L_NM_EN']}, "
        f"RLN_NAME_V1: {row['RLN_FM_NM_V1']} {row['RLN_L_NM_V1']}"
    )


In [ ]:
import pandas as pd
df=pd.read_excel("/content/Preprocessed_training_data.xlsx",engine="openpyxl")

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from sklearn.metrics import accuracy_score

# 1. Load and preprocess your data
#df = pd.read_excel("your_dataset.xlsx")  # Update with your filename

df["input_text"] = df.apply(format_input, axis=1)
df["target_text"] = df["FINAL SURNAME"].astype(str)

# 2. Create HuggingFace dataset
dataset = Dataset.from_pandas(df[["input_text", "target_text"]])
dataset = dataset.train_test_split(test_size=0.1)

# 3. Load tokenizer and model
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# 4. Tokenization
def preprocess_function(batch):
    model_inputs = tokenizer(batch["input_text"], max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["target_text"], max_length=16, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(preprocess_function, batched=True)

# 5. Define evaluation
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    pred_str = [p.strip() for p in pred_str]
    label_str = [l.strip() for l in label_str]
    return {"accuracy": accuracy_score(label_str, pred_str)}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/278421 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

Map:   0%|          | 0/30936 [00:00<?, ? examples/s]

In [ ]:
# 6. Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5_caste_model",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
)

# 7. Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 8. Train and Save
trainer.train()

model_path = "./mt5_caste_finetuned"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


/tmp/ipython-input-5-2039307294.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arpitshourya2233 (arpitshourya2233-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.003800,0.002248,0.994214
2,0.001500,0.001393,0.996412
3,0.001400,0.001035,0.996768
4,0.001000,0.000916,0.997026
5,0.001200,0.000899,0.997220


('./mt5_caste_finetuned/tokenizer_config.json',
 './mt5_caste_finetuned/special_tokens_map.json',
 './mt5_caste_finetuned/spiece.model',
 './mt5_caste_finetuned/added_tokens.json')

In [ ]:
import shutil

shutil.make_archive("mt5_caste_finetuned", 'zip', "mt5_caste_finetuned")

'/content/mt5_caste_finetuned.zip'

In [ ]:
import zipfile

with zipfile.ZipFile("mt5_caste_finetuned.zip", 'r') as zip_ref:
    zip_ref.extractall("mt5_caste_finetuned")


In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model_path = "mt5_caste_finetuned"  # path to unzipped folder

tokenizer = MT5Tokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path)


In [ ]:
def predict_surname(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    output = model.generate(**inputs, max_length=16)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example use
# sample_input = format_input(df.iloc[0])  # Or any new row
# print(predict_surname(sample_input))


In [ ]:
input_text = (
    "FM_NAME_EN: Rajesh, LASTNAME_EN: Verma, "
    "FM_NAME_V1: राजेश, LASTNAME_V1: वर्मा, "
    "RLN_TYPE: F, RLN_NAME_EN: Sita Devi, RLN_NAME_V1: सीता देवी"
)

print(predict_surname(input_text))  # Output: वर्मा


वर्मा
